###### Получаем базу знаний (просто папка нормативной документации из предоставленных данных)

In [5]:
!pip install gdown
!gdown 1vfU6ke9S-NCP9lo-164uYYZy7RNb1ZRl

Downloading...
From: https://drive.google.com/uc?id=1vfU6ke9S-NCP9lo-164uYYZy7RNb1ZRl
To: /home/user1/notebooks/knowlegebase.zip
100%|██████████████████████████████████████| 2.63M/2.63M [00:00<00:00, 21.8MB/s]


In [6]:
#!sudo apt install zip
!unzip knowlegebase.zip

Archive:  knowlegebase.zip
   creating: knowlegebase/
  inflating: knowlegebase/metodika_provedenija_inventarizacii_vybrosov_zagrj.md  
  inflating: knowlegebase/utochn-metodika-metalloobrabotka-2021.md  
  inflating: knowlegebase/ГОСТ Р 58577-2019. Национальный стандарт Российской Федераци.md  
  inflating: knowlegebase/Закон от 04_05_1999 N 96-ФЗ Об охране атмосферного воздуха (с изменениями на 8 августа 2024 года)_Текст.md  
  inflating: knowlegebase/Методика горных работ, Люберцы 1999.md  
  inflating: knowlegebase/Методика определения выбросов загрязняющих веществ в атмосферу при сжигании топлива в котлах, Москва, 1999.md  
  inflating: knowlegebase/Методика определения выбросов при сжигании топлива в котлах менее 20гкал в час.md  
  inflating: knowlegebase/Методика по нормированию и определению выбросов загрязняющих веществ в атмосферу на предприятиях нефтепродуктообеспечения ОАО.md  
  inflating: knowlegebase/Методика проведения инвентаризации выбросов для автотранспорт.md  
  i

## Начнем с QA модели

In [66]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain_core.documents import Document

from transformers import AutoTokenizer
from vllm import LLM, SamplingParams
from tqdm.auto import tqdm

import pandas as pd

Подразумевается что текущий ноутбук и папка test_dataset_iktin_test на одном уровне

In [10]:
test_df = pd.read_csv('test_dataset_iktin_test/Иктин test/Данные для тестирования/test.csv', sep='\t')

In [16]:
# Читаем директорию с нормативной документацией
loader = DirectoryLoader(
    './knowlegebase/',
    glob="./*.md",
    loader_cls=TextLoader,
    show_progress=True,
    use_multithreading=True,
)

documents = loader.load()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:00<00:00, 388.02it/s]


In [19]:
# готовим тексты для поиска через bm25
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=100
)
texts = text_splitter.split_documents(documents)

In [21]:
# загружаем модель для эмбедингов текста и создаем векторную базу данных для поиска
embeddings = HuggingFaceBgeEmbeddings(
    model_name='TatonkaHF/bge-m3_en_ru',
    model_kwargs={"device": "cuda"}
)
vectordb = FAISS.from_documents(
    documents = texts,
    embedding = embeddings
)

#### Используем модель phi3, длина контекста и размера кэша понижены, чтобы помещаться в видеопамять (tesla v100 32gb)

In [ ]:
MODEL_NAME = "microsoft/Phi-3-mini-128k-instruct"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
llm = LLM(model=MODEL_NAME, dtype='half', max_num_seqs=1, gpu_memory_utilization=0.8, max_model_len=25000)

##### Создание ретриверов bm25 и bge работают вместе для повышения качетсво поиска

In [22]:
k = 10
retriever_base = BM25Retriever.from_documents(texts, k=k//2)
retriever_advanced = vectordb.as_retriever(search_kwargs={"k": k//2, "search_type": "similarity"})
ensemble_retriever = EnsembleRetriever(retrievers=[retriever_base, retriever_advanced], weights=[0.5, 0.5], k=k)

In [25]:
# Небольшая настройка параметров для понижения галюцинаций системы
sampling_params = SamplingParams(temperature=0.01, repetition_penalty=0.8, frequency_penalty=1.2, max_tokens=2048, min_tokens=10, top_k=1)

##### Для начала обработаем общие вопросы не требующие файлов

In [28]:
questions = test_df[test_df['Документ']=='Нет']['Вопрос']

In [32]:
ans = {}

In [ ]:
for i, question in tqdm(enumerate(questions), total=len(questions)):
    retriever_ans_raw = ensemble_retriever.invoke(question, k=k)
    page_content = [i.page_content for i in retriever_ans_raw]
    retriever_ans = '\n\n'.join(page_content)

    prompt = tokenizer.apply_chat_template([{
        "role": "system",
        "content": f"""
        Твоя задача отвечать на вопросы пользователей связанные с экологией,
        тебе будет предоставлен контекст ответчай строго по нему.

        Контекст из которого необходимо взять информацию:
        {retriever_ans}

        ИНФОРМАЦИЮ ДЛЯ ОТВЕТА БЕРИ ТОЛЬКО ИЗ КОНТЕКСТА ВЫШЕ, ЕСЛИ ЕЕ ТАМ НЕТ НАПИШИ ИНФОРМАЦИЯ ОТСУТВУЕТ
        """
    }, {
        "role": "user",
        "content": question,
    }], tokenize=False, add_generation_prompt=True)

    output = llm.generate(prompt, sampling_params)
    ans[i+1] = output[0].outputs[0].text

##### Очистка файлов для ответов на конкретные вопросы

In [41]:
questions_with_files = test_df[test_df['Документ']!='Нет'][['Вопрос', 'Документ']]

In [ ]:
# бибилотеку необходимо установить 
# sudo apt libreoffice

##### Класс реализовывает в себе

In [62]:
import re
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTChar, LTFigure
from PyPDF2 import PdfReader, PdfWriter
from pdf2image import convert_from_path
import pdfplumber
import pytesseract
from PIL import Image
from datetime import datetime, timedelta
import os
from pathlib import PurePosixPath

class PDFScanner: # Класс отвечает за сканиорвание ПДФ-Документов
    CATEGORY_UPPER_BOUND = 3000
    NAME_UPPER_BOUND = 3000
    PATTERNS = {
        r"[\d,]*\n|\n.{1,2}\n": r"\n",
        r"\n[\d,.=]{1,10}": r"\n",
        r"^\s*\S*[^|]\s*$": r"",
        r"": "",
        r" *\n+": r"\n"
    }

    @staticmethod
    def scan_full(path) -> tuple[str, str, str]: # Скан документа с выдержкой краткого названия и категории
        pdf = PDFScanner.convert(path)
        text = PDFScanner.extract_text(pdf)
        category = categorize(text[0:PDFScanner.CATEGORY_UPPER_BOUND])
        delay = datetime.now() + timedelta(seconds=2)
        while datetime.now() < delay:
            pass
        name = rename(text[0:PDFScanner.NAME_UPPER_BOUND])
        return (category, name, text)

    @staticmethod
    def scan_partial(path) -> str: # Частичное сканирование документа
        pdf = PDFScanner.convert(path)
        text = PDFScanner.extract_text(pdf)
        return text
        
    @staticmethod
    def clean_text(text) -> str: # Очистить текст от артефактов
        for i in PDFScanner.PATTERNS:
            text = re.sub(i, PDFScanner.PATTERNS[i], text, flags=re.MULTILINE)
        return text
    
    @staticmethod
    def extract_text(path) -> str: # Получение текста и таблиц
        pdf_read = PdfReader(path)
        pages_dict = {} 
        for pagenum, page in enumerate(extract_pages(path)): # Проходимся по каждой странице
            page_obj = pdf_read.pages[pagenum]
            text_from_tables = []
            page_content = []
            table_in_page= -1
            pdf = pdfplumber.open(path)
            page_tables = pdf.pages[pagenum]

            tables = page_tables.find_tables() # Найти кол-во таблиц на странице
            if len(tables)!=0:
                table_in_page = 0

            # Извлечение таблиц
            for table_num in range(len(tables)):
                table = PDFScanner.extract_table(path, pagenum, table_num)
                table_string = PDFScanner.table_converter(table)
                text_from_tables.append(table_string)

            # Поиск элементов страницы
            page_elements = [(element.y1, element) for element in page._objs]
            # Сортировка в порядке их появления
            page_elements.sort(key=lambda a: a[0], reverse=True)


            scan_images = True # Сканировать картинки OCR'ом или нет
            for i, component in enumerate(page_elements): # Проходимся по каждому компоненту страницы
                element = component[1]

                # Check the elements for tables
                if table_in_page == -1:
                    pass
                else:
                    if PDFScanner.is_element_inside_any_table(element, page ,tables):
                        table_found = PDFScanner.find_table_for_element(element,page ,tables)
                        if table_found == table_in_page and table_found != None:    
                            table_in_page+=1
                        continue

                if not PDFScanner.is_element_inside_any_table(element,page,tables):
                    
                    # Если элемент - текстовое поле
                    if isinstance(element, LTTextContainer):
                        line_text = PDFScanner.extract_line(element)
                        # Append the format for each line containing text
                        page_content.append(line_text)
                        scan_images = False # Если увидели текст, то больше не сканируем картинки
                    continue
                    # Check the elements for images
                    if  scan_images and isinstance(element, LTFigure):
                        print("Scanning Image...")
                        # Crop the image from PDF
                        PDFScanner.crop_image(element, page_obj)
                        # Convert the croped pdf to image
                        PDFScanner.convert_to_images('cropped_image.pdf')
                        # Extract the text from image
                        image_text = PDFScanner.image_to_text('PDF_image.png')
                        page_content.append(image_text)
                        # Add a placeholder in the text and format lists

            # Create the key of the dictionary          
            dctkey = pagenum
            # Add the list of list as value of the page key
            pages_dict[dctkey]= [text_from_tables, page_content]

        md = ""
        for i in range(len(pages_dict)):
            for j in range(len(pages_dict[i][1])): # Добавление блоков текста
                md += PDFScanner.clean_text(pages_dict[i][1][j]) + '\n'
            for j in range(len(pages_dict[i][0])): # Добавление блоков таблиц
                md += pages_dict[i][0][j] + '\n'
        md = re.sub(r" *\n+", PDFScanner.PATTERNS[r" *\n+"], md, flags=re.MULTILINE)
        return md

    # Create a function to crop the image elements from PDFs
    @staticmethod
    def crop_image(element, pageObj):
        # Get the coordinates to crop the image from PDF
        [image_left, image_top, image_right, image_bottom] = [element.x0,element.y0,element.x1,element.y1] 
        # Crop the page using coordinates (left, bottom, right, top)
        pageObj.mediabox.lower_left = (image_left, image_bottom)
        pageObj.mediabox.upper_right = (image_right, image_top)
        # Save the cropped page to a new PDF
        cropped_pdf_writer = PdfWriter()
        cropped_pdf_writer.add_page(pageObj)
        # Save the cropped PDF to a new file
        with open('cropped_image.pdf', 'wb') as cropped_pdf_file:
            cropped_pdf_writer.write(cropped_pdf_file)

    # Create a function to convert the PDF to images
    @staticmethod
    def convert_to_images(input_file,):
        images = convert_from_path(input_file)
        image = images[0]
        output_file = 'PDF_image.png'
        image.save(output_file, 'PNG')

    # Create a function to read text from images
    @staticmethod
    def image_to_text(image_path):
        # Read the image
        img = Image.open(image_path)
        # Extract the text from the image
        text = pytesseract.image_to_string(img, lang='rus')
        return text

    # Create function to extract text
    @staticmethod
    def extract_line(element):
        # Extracting the text from the in line text element
        line_text = element.get_text()
        return line_text

    @staticmethod
    def extract_table(pdf_path, page_num, table_num):
        # Open the pdf file
        pdf = pdfplumber.open(pdf_path)
        # Find the examined page
        table_page = pdf.pages[page_num]
        # Extract the appropriate table
        table = table_page.extract_tables()[table_num]
        return table

    # Convert table into appropriate fromat
    @staticmethod
    def table_converter(table):
        # table_string = '\n'
        table_string = ''
        # Iterate through each row of the table
        for row_num in range(len(table)):
            row = table[row_num]
            # Remove the line breaker from the wrapted texts
            cleaned_row = [item.replace('\n', ' ') if item is not None and '\n' in item else '' if item is None else item for item in row]
            if row_num == 2:
                cleaned_row = ['-' for _ in row] 
                table_string+=('|'+'|'.join(cleaned_row)+'|'+'\n')
                continue
            if any(cleaned_row):
                # Convert the table into a string 
                table_string+=('|'+'|'.join(cleaned_row)+'|'+'\n')
        # Removing the last line break
        table_string = table_string[:-1]
        return table_string

    # Create a function to check if the element is in any tables present in the page
    @staticmethod
    def is_element_inside_any_table(element, page ,tables):
        x0, y0up, x1, y1up = element.bbox
        # Change the cordinates because the pdfminer counts from the botton to top of the page
        y0 = page.bbox[3] - y1up
        y1 = page.bbox[3] - y0up
        for table in tables:
            tx0, ty0, tx1, ty1 = table.bbox
            if tx0 <= x0 <= x1 <= tx1 and ty0 <= y0 <= y1 <= ty1:
                return True
        return False

    # Function to find the table for a given element
    @staticmethod
    def find_table_for_element(element, page ,tables):
        x0, y0up, x1, y1up = element.bbox
        # Change the cordinates because the pdfminer counts from the botton to top of the page
        y0 = page.bbox[3] - y1up
        y1 = page.bbox[3] - y0up
        for i, table in enumerate(tables):
            tx0, ty0, tx1, ty1 = table.bbox
            if tx0 <= x0 <= x1 <= tx1 and ty0 <= y0 <= y1 <= ty1:
                return i  # Return the index of the table
        return None
    
    @staticmethod
    def convert(path):
        if PurePosixPath(path).stem == "pdf":
            return path
        os.system(f"libreoffice --headless --convert-to pdf \"{path}\"")
        return PurePosixPath(path).stem + ".pdf"

In [59]:
pathqa = 'test_dataset_iktin_test/Иктин test/Данные для тестирования/Проект 2 (для QnA)/Проект Word/'
book1 = pathqa + 'Том 1 Инвентаризация Эко Агро.docx'
book2 = pathqa + 'Том 2 ПДВ Эко Агро.docx'

In [ ]:
!mkdir test_dataset_iktin_test/process

In [63]:
# обработка длится долго из-за чтения таблиц в markdown (до 5 минут из-за локльного перевода)
book1_text = PDFScanner.scan_partial(book1)
with open("test_dataset_iktin_test/process/book1_qa.md", "w", encoding="utf-8") as file:
    file.write(book1_text)

book2_text = PDFScanner.scan_partial(book2)
with open("test_dataset_iktin_test/process/book2_qa.md", "w", encoding="utf-8") as file:
    file.write(book2_text)

convert /home/user1/notebooks/test_dataset_iktin_test/Иктин test/Данные для тестирования/Проект 2 (для QnA)/Проект Word/Том 1 Инвентаризация Эко Агро.docx -> /home/user1/notebooks/Том 1 Инвентаризация Эко Агро.pdf using filter : writer_pdf_Export
convert /home/user1/notebooks/test_dataset_iktin_test/Иктин test/Данные для тестирования/Проект 2 (для QnA)/Проект Word/Том 2 ПДВ Эко Агро.docx -> /home/user1/notebooks/Том 2 ПДВ Эко Агро.pdf using filter : writer_pdf_Export


###### На сайте и в fastapi все реализованно через сессии, здесь в силу не хватки времени обработаем отдельно

In [68]:
book1_doc = Document(
    page_content=book1_text,
    metadata={"source": 'book1'}
)

texts_book1 = text_splitter.split_documents([book1_doc])

vectordb_book1 = FAISS.from_documents(
    documents = texts_book1,
    embedding = embeddings
)

In [91]:
k = 10
retriever_base_book1 = BM25Retriever.from_documents(texts_book1, k=k//2)
retriever_advanced_book1 = vectordb_book1.as_retriever(search_kwargs={"k": k//2, "search_type": "similarity"})
ensemble_retriever_book1 = EnsembleRetriever(retrievers=[retriever_base_book1, retriever_advanced_book1], weights=[0.5, 0.5], k=k)

In [72]:
book2_doc = Document(
    page_content=book1_text,
    metadata={"source": 'book2'}
)

texts_book2 = text_splitter.split_documents([book2_doc])

vectordb_book2 = FAISS.from_documents(
    documents = texts_book2,
    embedding = embeddings
)

In [92]:
k = 10
retriever_base_book2 = BM25Retriever.from_documents(texts_book2, k=k//2)
retriever_advanced_book2 = vectordb_book2.as_retriever(search_kwargs={"k": k//2, "search_type": "similarity"})
ensemble_retriever_book2 = EnsembleRetriever(retrievers=[retriever_base_book2, retriever_advanced_book2], weights=[0.5, 0.5], k=k)

In [ ]:
for i, info in tqdm(questions_with_files.iterrows(), total=len(questions_with_files)):
    doc = info['Документ']
    question = info['Вопрос']

    if doc.startswith('Книга 1'):
        retriever_ans_raw = ensemble_retriever_book1.invoke(question, k=k)
    else:
        retriever_ans_raw = ensemble_retriever_book2.invoke(question, k=k)

    page_content = [i.page_content for i in retriever_ans_raw]
    retriever_ans = '\n\n'.join(page_content)

    prompt = tokenizer.apply_chat_template([{
        "role": "system",
        "content": f"""
        Твоя задача отвечать на вопросы пользователей связанные с экологией,
        тебе будет предоставлен контекст ответчай строго по нему.

        Контекст из которого необходимо взять информацию:
        {retriever_ans}

        ИНФОРМАЦИЮ ДЛЯ ОТВЕТА БЕРИ ТОЛЬКО ИЗ КОНТЕКСТА ВЫШЕ, ЕСЛИ ЕЕ ТАМ НЕТ НАПИШИ ИНФОРМАЦИЯ ОТСУТВУЕТ
        """
    }, {
        "role": "user",
        "content": question,
    }], tokenize=False, add_generation_prompt=True)

    output = llm.generate(prompt, sampling_params)
    ans[i+1] = output[0].outputs[0].text

In [111]:
df = pd.DataFrame(list(ans.items()), columns=['№', 'answer'])

df['answer'] = df['answer'].str.replace('\n', '', regex=False) # убираем \n потому что модель отвечает в markdown

In [118]:
df.to_csv('submit.csv', index=False, sep='\t')

## Суммаризация

На сайте реализована суммаризация всех подгружаемых файлов, подробнее можно ознакомится там

Из-за особенностей библиотеки pytesseract, которая требует явно указать путь к библиотеке, аналогично для windows pytesseract.pytesseract.tesseract_cmd = 'C:\Program Files (x86)\Tesseract-OCR\tesseract.exe'. Еще поддержка есть на нашем fastapi

In [121]:
!gdown 1lsw-3qYL6DTEG2ut1WI3nVY-dTCDmnu_
!gdown 1r23qz36CNy6PJVWbDRgCVJp6Hmixo3wj

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Downloading...
From: https://drive.google.com/uc?id=1lsw-3qYL6DTEG2ut1WI3nVY-dTCDmnu_
To: /home/user1/notebooks/Книга_1_Инвентаризация_Био_Агро_Дон_1.pdf
100%|██████████████████████████████████████| 22.8M/22.8M [00:00<00:00, 73.7MB/s]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Downloading...
From: https://drive.google.com/uc?id=1r23qz36CNy6PJVWbDRgCVJp6Hmixo3wj
To: /home/user1/notebooks/Книга 1 ПДВ Био Агро Дон.pdf
100%|██████████████████████████████████████| 12.5M/12.5M [00:00<00:00, 50.4MB/s]


Суммаризация в отличае от других частей нашего проекта работает через YandexGPT. Ниже приведен код который поможет получить эту суммаризцию. Вы можете добавить свои ключи или перейти на наш сайт http://iktin.tw1.su/docs/ по этому адресу в категории Тест находятся файлы предложеные нам для суммаризации

In [ ]:
import requests
import os
from dotenv import load_dotenv

SYSTEM_SUM = """
Шаблон структуры характеристик предприятия
1. Основной вид деятельности
2. Основное структурное подразделение
3. Процесс разработки
4. Склады хранения
5. Заправка техники
6. Выбросы загрязняющих веществ (ЗВ)

Проанализируй текст ниже и просуммаризируй его, составив параграф, структура которого приведена выше. Старайся меньше придумывать и больше придерживаться текста документа.
"""
SUMMARIZER = YA_GPT(SYSTEM_SUM)


def summarize(text):
    return SUMMARIZER.query(text)

load_dotenv()
IAM_TOKEN = os.getenv("IAM_TOKEN")
FOLDER_ID = os.getenv("FOLDER_ID")


class YA_GPT():
    URL = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
    IAM_TOKEN = os.getenv("IAM_TOKEN")
    FOLDER_ID = os.getenv("FOLDER_ID")

    def __init__(self, system) -> None:
        self.system = system
        self.headers = {
            "Content-Type": "application/json",
            "Authorization": f"Api-Key {YA_GPT.IAM_TOKEN}",
        }

    def query(self, prompt):
        query = {
            "modelUri": f"gpt://{YA_GPT.FOLDER_ID}/yandexgpt-32k/rc",
            "completionOptions": {
                "stream": False,
                "temperature": 0.6,
                "maxTokens": "2000"
            },
            "messages": [
                {
                "role": "system",
                "text": self.system
                },
                {
                "role": "user",
                "text": prompt
                }
            ]
        }

        response = requests.post(YA_GPT.URL, headers=self.headers, json=query).json()
        print(response)
        return response["result"]["alternatives"][0]["message"]["text"]

summarize(text)